<h3 align="center">
实验二：磁性材料基本特性研究
</h3>

请将第一个 python 单元格中的内容复制，**和你的实验报告的完整照片**提交给一个大语言模型（我们建议使用 Google Gemini 2.5 Pro 或者等价逻辑能力的LLM），并用生成的 python 代码块替换该单元格中的实例数据点。

In [ ]:
'''
You are a lab agent, and your task is to extract data faithfully from the images provided. 
If no image is provided, please tell the user to upload photos of his or her lab report.
Read the following instructions carefully before you answer.
- The images should contain clear, handwritten text. If any part is unclear, ask the user to supply a clearer image or transcribe the unclear part.
- Pay special attention to the unit used by the user. compare measure results with standard ones, and convert them if necessary. Automatically adjust magnitudes, and inform the user of the adjustments in text before the python snippet.
- When writing code you are expected to follow the given format strictly. If no warnings or failures occur, you should output a single python snippet, enclosed within triple backticks. You need not add extra comments to your answer.
- You need not output the prepending requirements, only the data extraction result in python code format.

---

Extract the date of the experiment to `date`. If it is not present, ask the user to provide it before trying to extract data again.

---

> Name of Experiment 1: Measurement of Saturation Hysteresis Loop (磁滞回线的测量)

In Experiment 1, you need to extract the following data:
- A table of voltage readings for UR1 data and Uc data;
- Parameters of equipments: R_1, R_2, C_1 data;

Please convert the data into SI units (use mV for the table). Read the sample data below for approximate value after conversion.

---

> Name of Experiment 2: Measurement of Curie Temperature (居里温度的测量)

In Experiment 2, you need to extract the following data:

A table mapping the temperature T (°C) to measured voltage Uout (mV). The temperature should be increasing and the voltage decreasing.

Note that the data may have multiple lines. You need to extract all the data.

Handwriting may be unclear in this part. Infer from the data trend if you can, and ask for clarification if you cannot. The two measurements should on the large scale be monotonic.
'''

# TODO: Replace the sample data below with the extracted data.

date = "2025-01-01"  # Example date, replace with actual extracted date

# Experiment 1: Measurement of Saturation Hysteresis Loop
# 实验一：饱和磁滞回线的测量
UR1 = [-217.6, -133.1, -106.2, -69.12, -34.56, -12.80] # unit: mV
Uc = [-80.33, -71.82, -67.03, -60.65, -52.14, -45.12]  # unit: mV

R1 = 1e1          # unit: ohms
R2 = 1e3          # unit: ohms
C1 = 5e-6         # unit: farads

# Experiment 2: Measurement of Curie Temperature
# 实验二：测量居里温度
T = [25.6, 30.0, 35.0, 40.0, 86.2, 86.7, 87.5, 88.7] # unit: degree Celsius
Uout = [23.888, 23.880, 23.861, 23.844, 4.577, 4.070, 3.550, 3.091] # unit: mV

In [ ]:
# Data analysis for Experiment 1
# 实验一数据处理

L = 3.61 * 10**(-2)
S = 1.25 * 10**(-5)
N1 = 100
N2 = 100

assert len(UR1) == len(Uc), "UR1 data and Uc data must have the same length. If this fails, check the extracted data."
cnt_data = len(UR1)

# convert mV to V
Uc = [u * 1e-3 for u in Uc]
UR1 = [u * 1e-3 for u in UR1]

# calc B and H
B = [Uc[i] * R2 * C1 / N2 / S for i in range(cnt_data)]
H = [UR1[i] * N1 / R1 / L for i in range(cnt_data)]


In [ ]:
# Create a table to display the results
# 显示结果表格

from rich.console import Console
from rich.table import Table

table = Table(show_header=True)
table.add_column("UR1 (mV)")
table.add_column("Uc (mV)")
table.add_column("B (T)")
table.add_column("H (A/m)")

for i in range(cnt_data):
    table.add_row(f"{UR1[i] * 1000:.2f}", f"{Uc[i] * 1000:.2f}", f"{B[i]:.4f}", f"{H[i]:.4f}")

Console().print(table)


In [ ]:
# Use matplotlib to save the table to disk
# 使用 matplotlib 将表格保存到磁盘

import matplotlib.pyplot as plt
import graphing # noqa
import os

# TODO 更改 figsize 以适应表格大小
fig, ax = plt.subplots(figsize=(8, 11))
fig.suptitle('实验一：饱和磁滞回线测量结果表格', fontsize=16)
ax.axis('off')

tbl = plt.table(
    cellText=[[f"{UR1[i] * 1000:.2f}", f"{Uc[i] * 1000:.2f}", f"{B[i]:.4f}", f"{H[i]:.4f}"] for i in range(cnt_data)],
    colLabels=["UR1 (mV)", "Uc (mV)", "B (T)", "H (A/m)"],
    cellLoc='center',
    loc='center',
)
tbl.auto_set_font_size(False)
tbl.set_fontsize(10)
tbl.scale(1, 1.5)
os.makedirs("output", exist_ok=True)
plt.savefig("output/experiment1_results_table.png", bbox_inches='tight', dpi=300)
plt.show()

In [ ]:

# Sort and rearrange the data for plotting
# 排序并重新排列数据以便绘图

assert(isinstance(H, list) and isinstance(B, list) and len(H) == len(B)), "H and B must be lists of the same length."
data = [[H[i], B[i]] for i in range(len(B))]
data = sorted(data, key=lambda x: x[0])  # Sort by H

B1, H1, B2, H2 = [], [], [], []
B1.append(data[0][1])
H1.append(data[0][0])
# If the 2nd and 3rd points' H are similar, put the larger B in B1 and the smaller B in B2
# 假设第 2 3 个点的 H 类似，那么 B 大的放在 B1 小的放在 B2
if data[1][1] > data[2][1]:
    B1.append(data[1][1])
    H1.append(data[1][0])
    B2.append(data[2][1])
    H2.append(data[2][0])
else:
    B1.append(data[2][1])
    H1.append(data[2][0])
    B2.append(data[1][1])
    H2.append(data[1][0])
for i in range(3, len(data)):
    if data[i][1] > B1[-1]:
        B1.append(data[i][1])
        H1.append(data[i][0])
    else:
        B2.append(data[i][1])
        H2.append(data[i][0])

H, B = H1, B1
# Invert H2, B2 and append to H, B
# H2, B2 翻转之后加入 H, B
H += H2[::-1]
B += B2[::-1]


In [ ]:
# Output the image:
# If the curve is not correct, manually reorder the data points.
# 输出图像
# 如果曲线不对，那就手动修改一下点的顺序

import numpy as np
import matplotlib.pyplot as plt
from graphing import font # noqa
from graphing.utils import add_signature, add_grid

def move_point(ori, new, H_arr, B_arr):
    '''
    Move a point from its original index to a new index in NumPy arrays.
    NOTE: This function returns new arrays and does not modify the originals.
    将一个点从原来的索引移动到新的索引位置 (适用于 NumPy 数组)
    注意：此函数返回新的数组，不会修改原始数组。
    '''
    assert isinstance(ori, int) and isinstance(new, int), "ori and new must be integers."
    assert 0 <= ori < len(H_arr) and 0 <= new <= len(H_arr), "ori and new must be valid indices."

    # Store the values to be moved
    h_val = H_arr[ori]
    b_val = B_arr[ori]
    
    # Use np.delete to create new arrays without the element at 'ori'
    temp_H = np.delete(H_arr, ori)
    temp_B = np.delete(B_arr, ori)
    
    # Use np.insert to create final arrays with the element inserted at 'new'
    # The index adjustment is tricky if new > ori, but np.delete/insert handles this logic correctly.
    final_H = np.insert(temp_H, new, h_val)
    final_B = np.insert(temp_B, new, b_val)
    
    return final_H, final_B


# TODO 排列 H 和 B 直到曲线正确
'''
# Example Usage: Move 3 to after point of original index 36
H, B = move_point(1, 2, H, B)
'''
H = np.asarray(H, dtype=float)
B = np.asarray(B, dtype=float)

# Add a point to close the loop
# 添加一个点以闭合回线
H_plot = np.r_[H, H[0]]
B_plot = np.r_[B, B[0]]

fig, ax = plt.subplots(
    figsize=(8, 6)
)

ax.axhline(0, linewidth=1)
ax.axvline(0, linewidth=1)

ax.plot(H_plot, B_plot, linestyle='-', marker='s', linewidth=1, color='black')

# TODO 开启注释以显示点的索引
# Label each dot with its new index
# 为每个点标上新的索引
# for i in range(len(H_plot)):
#     ax.text(H_plot[i], B_plot[i], str(i), fontsize=12, verticalalignment='bottom', horizontalalignment='right')

ax.set_title('实验一：磁滞回线的测量和绘制', pad=10)

ax.set_xlabel('磁场强度 H (A/m)')
ax.set_ylabel('磁感应强度 B (T)')
add_signature(ax, date=date, position='lower right')

add_grid(ax, x=(25, 5), y=(0.2, 0.05))
ax.margins(x=0.05, y=0.05)

plt.savefig("output/experiment1_hysteresis_loop.png", bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter1d
from graphing.units import celsius

from graphing.utils import set_margin
set_margin(0, 0.2)

# Plot 2.1: Measurement of Curie Temperature
# 第一张图：电压与温度的关系

plt.figure(figsize=(10, 6))
plt.plot(T, Uout, marker='s', linestyle='-', color='black', markersize=4)

plt.title('实验二：输出电压和温度的关系', pad=10)
plt.xlabel(f'温度 T ({celsius})')
plt.ylabel('输出电压 Uout (mV)')

add_grid(plt.gca(), x=(5, 1), y=(5, 1))
add_signature(plt.gca(), date=date, position='lower left')

plt.savefig("output/experiment2_curie_temperature.png", bbox_inches='tight', dpi=300)
plt.show()

# Plot 2.2: Derivative of Voltage with respect to Temperature
# 第二张图：电压的导数与温度的关系

T = np.asarray(T, dtype=float)
Uout = np.asarray(Uout, dtype=float)

dU_dt = np.gradient(Uout, T)

# Smooth the data points
# 平滑处理
# TODO 调整 sigma 以获得更好的平滑效果
dU_dt = gaussian_filter1d(dU_dt, sigma=1.4)

# Find the point with the maximum absolute value
# 找到绝对值最大的点
max_index = 0
for i in range(1, len(dU_dt) - 1):
    if abs(dU_dt[i]) > abs(dU_dt[max_index]):
        max_index = i

plt.figure(figsize=(10, 6))
plt.plot(T, dU_dt, marker='s', linestyle='-', color='black', markersize=4)

# Annotate the max_index point on the graph
# 在图像上标注 max_index 点

plt.scatter(T[max_index], dU_dt[max_index], color='r', s=150, zorder=5, label='Max |dU/dT|', marker='o')

plt.annotate(f'变化率最高的温度（居里温度）\nT = {T[max_index]:.2f}{celsius}',
    xy=(
        T[max_index], dU_dt[max_index]
    ),
    xytext=(T[max_index] - 25, dU_dt[max_index] - 0.2), # Position of the text
    arrowprops=dict(facecolor='black', shrink=0.05, width=1, headwidth=6),
    fontsize=12,
    bbox=dict(boxstyle="round", fc="w", ec="0.5", alpha=0.9),
)
add_signature(plt.gca(), date=date, position='lower left')

plt.title('实验二：电压变化率和温度的关系 (dU/dT)', pad=10)
plt.xlabel(f'温度 T ({celsius})')
plt.ylabel(f'电压变化率 dU/dT (mV/{celsius})')
add_grid(plt.gca(), x=(5, 1), y=(1, 0.1))

plt.savefig("output/experiment2_curie_temperature_derivative.png", bbox_inches='tight', dpi=300)
plt.show()

### 附录的打印

运行完所有的程序后，你将在 `output/` 目录下找到生成的实验报告图像。你可以在 [Gridify](https://rayzh-hs.github.io/gridify/) 上排列这些图片，输出成 PDF 以便打印和提交。
